In [1]:
from manual_influencer import constants, script_generator

#mjson = script_generator.generate_script(text=constants.TEXT_AND_PDF_TO_SCENES_PROMPT)

In [2]:
import json
#data = json.loads(mjson)

In [3]:
from google import genai
from google.genai import types
import base64

def generate(prompt):
    client = genai.Client(
      vertexai=True,
      project="bliss-hack25fra-9587",
      location="global",
    )

    si_text1 = """make an english description of an image matching this scene description. black and white, rough sketch, cartoonish style. only answer with the description. no other text."""

    model = "gemini-2.0-flash-001"
    contents = [
        types.Content(
          role="user",
          parts=[
            types.Part.from_text(text=prompt)
          ]
        ),
    ]
    generate_content_config = types.GenerateContentConfig(
        temperature = 1,
        top_p = 0.95,
        max_output_tokens = 8192,
        response_modalities = ["TEXT"],
        safety_settings = [types.SafetySetting(
          category="HARM_CATEGORY_HATE_SPEECH",
          threshold="OFF"
        ),types.SafetySetting(
          category="HARM_CATEGORY_DANGEROUS_CONTENT",
          threshold="OFF"
        ),types.SafetySetting(
          category="HARM_CATEGORY_SEXUALLY_EXPLICIT",
          threshold="OFF"
        ),types.SafetySetting(
          category="HARM_CATEGORY_HARASSMENT",
          threshold="OFF"
        )],
        system_instruction=[types.Part.from_text(text=si_text1)],
    )

    mstr = ''

    for chunk in client.models.generate_content_stream(
        model = model,
        contents = contents,
        config = generate_content_config,
        ):
        print(chunk.text, end="")
        mstr += chunk.text

    return mstr
    

#generate()

In [4]:
from vertexai.preview.vision_models import ImageGenerationModel
import vertexai

#pmt = "A close-up, roughly sketched, black and white cartoon drawing of a powerline adapter, suggesting a simple, perhaps technical illustration."
def generate_image(prompt, filename):
    print("Prompt: " + prompt)
    vertexai.init(project="bliss-hack25fra-9587", location="us-central1")
    generation_model = ImageGenerationModel.from_pretrained("imagen-3.0-generate-002")

    images = generation_model.generate_images(
        prompt=prompt,
        number_of_images=1,
        aspect_ratio="9:16",
        negative_prompt="",
        person_generation="",
        safety_filter_level="",
        add_watermark=True,
    )

    images[0].save(filename)

In [5]:
import cv2
import os
import subprocess
import imageio_ffmpeg
import ffmpeg

def generate_video(image_folder = 'data/images', audio_file = './data/synthesis.wav', output_file = './data/out.mp4'):
    try:
        os.remove('./finished_video.mp4')
        os.remove('./video.avi')
        os.remove('./video.mp4')
    except:
        pass

    images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter('./video.avi', 0, 0.25, (width,height))

    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    #cv2.destroyAllWindows()
    video.release()



    ffmpeg_exe = imageio_ffmpeg.get_ffmpeg_exe()
    subprocess.run([
        ffmpeg_exe,
        '-y',               # overwrite output if exists
        '-i', 'video.avi',  # input video
        '-c:v', 'libx264',  # video codec
        '-c:a', 'aac',      # audio codec
        '-shortest',        # finish when the shorter stream ends
        'video.mp4'         # output file
    ])
    
    subprocess.run([
        ffmpeg_exe,
        '-y',
        '-i', 'video.mp4',
        '-i', audio_file,
        '-c:v', 'copy',
        '-c:a', 'aac',
        '-shortest',
        output_file
    ])

    #input_video = ffmpeg.input('./video.mp4')

    #input_audio = ffmpeg.input(audio_file)

    #ffmpeg.concat(input_video, input_audio, v=1, a=1).output(output_file).run(cmd=ffmpeg_exe)
    os.remove('./video.avi')
    os.remove('./video.mp4')

In [6]:
import subprocess
import json
import wave
import contextlib
from mutagen.mp3 import MP3

def get_audio_duration(filepath):
    audio = MP3(filepath)
    print(audio.info.length)
    return audio.info.length

In [7]:
import ffmpeg
import imageio_ffmpeg
import os

def concat_videos(video_paths, output_path='concatenated.mp4'):
    # Create the concat input text file
    with open('concat_list.txt', 'w') as f:
        for path in video_paths:
            f.write(f"file '{os.path.abspath(path)}'\n")

    ffmpeg_exe = imageio_ffmpeg.get_ffmpeg_exe()

    # Run ffmpeg concat
    subprocess.run([
        ffmpeg_exe,
        '-f', 'concat',
        '-safe', '0',
        '-i', 'concat_list.txt',
        '-c', 'copy',
        output_path
    ])

    os.remove('concat_list.txt')

In [ ]:
from manual_influencer.tts import synthesize_speech
import shutil
from pathlib import Path
import os

for mp4_file in Path('./vids/').glob('*.mp4'):
    try:
        os.remove(mp4_file)
    except:
        pass

for k in range(1, 50):
    try:
        mjson = script_generator.generate_script(text=constants.TEXT_AND_PDF_TO_SCENES_PROMPT)
        data = json.loads(mjson)
        
        folder = './data/visual'
        for f in os.listdir(folder):
            path = os.path.join(folder, f)
            if os.path.isfile(path):
                os.remove(path)

        try:
            os.remove('./concatenated.mp4')
        except:
            pass

        for mp4_file in Path('.').glob('*.mp4'):
            try:
                os.remove(mp4_file)
            except:
                pass

        images = []
        idx = 0

        for scene in data['scenes']:
            print(scene)
            visual = "A roughly sketched, cartoonish black and white image with no text; "
            visual += generate(scene['visual'])
            synthesize_speech(text=scene['audio'], output_file='./data/audio/output.mp3')
            time = get_audio_duration('./data/audio/output.mp3')
            for i in range(1, int(time/4+2)):
                generate_image(visual, './data/visual/images_00' + str(i) + '.png')
            generate_video(image_folder='./data/visual', audio_file='./data/audio/output.mp3', output_file = str(idx) + '.mp4')
            for f in os.listdir(folder):
                path = os.path.join(folder, f)
                if os.path.isfile(path):
                    os.remove(path)
            idx += 1

        ml = []

        for mp4_file in Path('.').glob('*.mp4'):
            ml.append(mp4_file)

        print('Creating vid: ' + str(k))
        concat_videos(ml, output_path='./vids/vid_' + str(k) + '.mp4')
    except:
        pass

{
  "scenes": [
    {
      "visual": "Ein PowerLine Webcast 3 Adapter wird in eine Steckdose gesteckt.",
      "audio": "Bevor Sie die TechniSat PowerLine Webcast 3 Adapter in Betrieb nehmen, prüfen Sie zunächst die Lieferung auf Vollständigkeit."
    },
    {
      "visual": "Ein Benutzer versucht, den Adapter zu öffnen.",
      "audio": "Das Gerät darf nicht geöffnet werden! Es befinden sich keine durch den Anwender zu wartenden Teile im Inneren des Gerätes."
    },
    {
      "visual": "Der Adapter steht in direkter Sonneneinstrahlung.",
      "audio": "Setzen Sie das Gerät nicht direkter Sonneneinstrahlung aus."
    },
    {
      "visual": "Ein defekter Adapter wird an TechniSat gesendet.",
      "audio": "Für eine eventuell erforderliche Einsendung des Gerätes verwenden Sie bitte ausschliesslich folgende Anschrift: TechniSat Digital GmbH, Service-Center."
    }
  ]
}{'visual': 'Ein PowerLine Webcast 3 Adapter wird in eine Steckdose gesteckt.', 'audio': 'Bevor Sie die TechniSat 

ffmpeg version 7.0.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libdav1d --enable-libxvid --enable-libzvbi --enable-libzimg
  libavutil      59.  8.100 / 59.  8.100
  libavcodec     61.  3.100 / 61.  3.100
  libavformat   

{'visual': 'Ein Benutzer versucht, den Adapter zu öffnen.', 'audio': 'Das Gerät darf nicht geöffnet werden! Es befinden sich keine durch den Anwender zu wartenden Teile im Inneren des Gerätes.'}
A roughly sketched cartoon of a person struggling to open an adapter, rendered in black and white with a slightly messy, hand-drawn aesthetic.
Audio content written to file "./data/audio/output.mp3"
7.752
Prompt: A roughly sketched, cartoonish black and white image with no text; A roughly sketched cartoon of a person struggling to open an adapter, rendered in black and white with a slightly messy, hand-drawn aesthetic.



ffmpeg version 7.0.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libdav1d --enable-libxvid --enable-libzvbi --enable-libzimg
  libavutil      59.  8.100 / 59.  8.100
  libavcodec     61.  3.100 / 61.  3.100
  libavformat   

{'visual': 'Der Adapter steht in direkter Sonneneinstrahlung.', 'audio': 'Setzen Sie das Gerät nicht direkter Sonneneinstrahlung aus.'}
A black and white, roughly sketched, cartoonish depiction of an adapter plug sitting in direct sunlight.
Audio content written to file "./data/audio/output.mp3"
3.72
{
  "scenes": [
    {
      "visual": "Person steckt PowerLine Webcast 3 Adapter in die Steckdose.",
      "audio": "Bevor Sie die TechniSat PowerLine Webcast 3 Adapter in Betrieb nehmen, schliessen Sie zunächst beide Geräte wie im Vorfeld beschrieben an. Jetzt läuft die Datenübertragung über das Stromnetz per Knopfdruck. Sie müssen den Verschlüsselungsknopf des zusätzlichen Adapters drücken, ansonsten wenden Sie sich bitte zunächst an unsere Technische Hotline."
    }
  ]
}{'visual': 'Person steckt PowerLine Webcast 3 Adapter in die Steckdose.', 'audio': 'Bevor Sie die TechniSat PowerLine Webcast 3 Adapter in Betrieb nehmen, schliessen Sie zunächst beide Geräte wie im Vorfeld beschrieben 

...

{'visual': 'Eine Power LED blinkt hektisch auf einem Adapter, während eine Person frustriert auf den Verschlüsselungsknopf hämmert.', 'audio': 'Wenn die Power LED aufgehört hat zu blinken und stattdessen dauerhaft leuchtet, ist die Verbindung geschützt. Falls nicht, wenden Sie sich bitte an unsere Technische Hotline, bevor Sie den Adapter aus dem Fenster werfen.'}
A power LED blinks erratically on an adapter while a person frustratingly mashes a button labeled "encryption".
Audio content written to file "./data/audio/output.mp3"
13.776
Prompt: A roughly sketched, cartoonish black and white image with no text; A power LED blinks erratically on an adapter while a person frustratingly mashes a button labeled "encryption".

Prompt: A roughly sketched, cartoonish black and white image with no text; A power LED blinks erratically on an adapter while a person frustratingly mashes a button labeled "encryption".

Prompt: A roughly sketched, cartoonish black and white image with no text; A power

ffmpeg version 7.0.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libdav1d --enable-libxvid --enable-libzvbi --enable-libzimg
  libavutil      59.  8.100 / 59.  8.100
  libavcodec     61.  3.100 / 61.  3.100
  libavformat   

Creating vid: 10
{
  "scenes": [
    {
      "visual": "Zeigt den PowerLine Webcast 3 Adapter.",
      "audio": "Ich wollte mein gesichertes Netzwerk erweitern, steckte den PowerLine Webcast 3 Adapter in die Steckdose."
    },
    {
      "visual": "Nahaufnahme einer Steckdose.",
      "audio": "Doch dann bemerkte ich, dass die Steckdose defekt ist!"
    },
    {
      "visual": "Zeigt eine Person, die verzweifelt versucht, ein Netzwerkkabel anzuschließen.",
      "audio": "Ich hab das Netzwerkkabel ordnungsgemäß angeschlossen. Und ich habe die Lieferung auf Vollständigkeit überprüft."
    },
    {
      "visual": "Eine Hand drückt den Verschlüsselungsknopf.",
      "audio": "Zum Schluss habe ich einfach den Verschlüsselungsknopf betätigt, um das Gerät aus dem bestehenden Netzwerk zu entfernen."
    }
  ]
}{'visual': 'Zeigt den PowerLine Webcast 3 Adapter.', 'audio': 'Ich wollte mein gesichertes Netzwerk erweitern, steckte den PowerLine Webcast 3 Adapter in die Steckdose.'}
A cartoonis

ffmpeg version 7.0.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libdav1d --enable-libxvid --enable-libzvbi --enable-libzimg
  libavutil      59.  8.100 / 59.  8.100
  libavcodec     61.  3.100 / 61.  3.100
  libavformat   

{'visual': 'Nahaufnahme einer Steckdose.', 'audio': 'Doch dann bemerkte ich, dass die Steckdose defekt ist!'}
A close-up of an electrical outlet.
Audio content written to file "./data/audio/output.mp3"
3.576
{
  "scenes": [
    {
      "visual": "Eine Person, die verwirrt auf einen Powerline Webcast 3 Adapter starrt.",
      "audio": "Bevor Sie diesen TechniSat PowerLine Webcast 3 Adapter in Betrieb nehmen, überprüfen Sie zunächst die Lieferung auf Vollständigkeit. Was ist, wenn das Gerät mit Flüssigkeit überschüttet wurde oder Gegenstände in das Gerät hineingeraten sind? Ich meine, wie wahrscheinlich ist das? Wenn die Power LED aufgehört hat zu blinken und stattdessen dauerhaft leuchtet, ist die Verbindung zwischen den beiden Adaptern geschützt."
    }
  ]
}{'visual': 'Eine Person, die verwirrt auf einen Powerline Webcast 3 Adapter starrt.', 'audio': 'Bevor Sie diesen TechniSat PowerLine Webcast 3 Adapter in Betrieb nehmen, überprüfen Sie zunächst die Lieferung auf Vollständigkeit. Wa

In [ ]:
from pathlib import Path

ml = []

#for mp4_file in Path('.').glob('*.mp4'):
#    ml.append(mp4_file)
    
#concat_videos(ml)